In [ ]:
#torch and lightning (deep learning/model creation and training)
import torch
from torch import nn
import lightning as L
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

#ray (tune/grid search)
from ray import train, tune, put, get, init
from ray.tune.search.optuna import OptunaSearch
from ray.tune.integration.pytorch_lightning import TuneReportCheckpointCallback
from ray.tune.schedulers import ASHAScheduler
from functools import partial


#matplotlib and seaborn (plotting)
import matplotlib.pyplot as plt
import seaborn as sns

# #pyOD (outlier detection)
# from pyod.models.iforest import IForest

#mlflow (loggind/tracking)
import mlflow

#general purpose
import numpy as np
import pandas as pd
import os

#split data into train and test sets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold


/home/pedro/Documents/dl4spl2/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-23 15:05:58,738	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-01-23 15:05:58,880	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
df = pd.read_parquet('data.parquet')
df.head(4)

,X86_LOCAL_APIC,OPENVSWITCH,TEXTSEARCH_FSM,NETFILTER_XT_MATCH_TCPMSS,MPLS,NFC_HCI,NETFILTER_XT_MATCH_TIME,NET_MPLS_GSO,NFC_SHDLC,NETFILTER_XT_MATCH_U32,...,ARCH_SUPPORTS_INT128,SLABINFO,MICROCODE_AMD,ISDN_DRV_HISAX,CHARGER_BQ24190,SND_SOC_NAU8825,BH1750,NETWORK_FILESYSTEMS,active_options,perf
0,1,0,0,0,1,0,0,1,0,0,...,1,0,0,0,1,0,0,0,1435,50222120
1,1,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,1382,16660024
2,1,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1626,43080856
3,1,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,0,1,0,2140,27261672


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92562 entries, 0 to 92561
Columns: 9469 entries, X86_LOCAL_APIC to perf
dtypes: int64(9469)
memory usage: 6.5 GB


# Float point transformation

+ Pass the data to float
+ Get used on pytorch models

In [ ]:
df = df.astype('float32')

In [ ]:
#df.describe()

In [6]:
df = df.sample(1500) # 1+ comente ou remova essa celula para rodar com o dataset completo, o tempo de execução será extenso.
df

X86_LOCAL_APIC  OPENVSWITCH  TEXTSEARCH_FSM  NETFILTER_XT_MATCH_TCPMSS  \
35113             1.0          0.0             0.0                        0.0   
12798             1.0          0.0             0.0                        0.0   
48733             1.0          0.0             0.0                        0.0   
36338             1.0          0.0             0.0                        0.0   
74920             1.0          0.0             0.0                        0.0   
...               ...          ...             ...                        ...   
30545             1.0          0.0             0.0                        0.0   
10020             1.0          0.0             0.0                        0.0   
61559             1.0          0.0             0.0                        0.0   
88900             1.0          0.0             0.0                        0.0   
21891             1.0          0.0             0.0                        0.0   

       MPLS  NFC_HCI  NETFILTER_XT_MATCH_TIME  NET_MPLS_GSO  NFC_SHDLC  \
35113   0.0      0.0                      0.0           0.0        0.0   
12798   0.0      0.0                      0.0           0.0        0.0   
48733   0.0      0.0                      0.0           0.0        0.0   
36338   0.0      0.0                      0.0           0.0        0.0   
74920   0.0      0.0                      0.0           0.0        0.0   
...     ...      ...                      ...           ...        ...   
30545   0.0      0.0                      0.0           0.0        0.0   
10020   0.0      0.0                      0.0           0.0        0.0   
61559   0.0      0.0                      0.0           0.0        0.0   
88900   0.0      0.0                      0.0           0.0        0.0   
21891   0.0      0.0                      0.0           0.0        0.0   

       NETFILTER_XT_MATCH_U32  ...  ARCH_SUPPORTS_INT128  SLABINFO  \
35113                     0.0  ...                   1.0       0.0   
12798                     0.0  ...                   1.0       1.0   
48733                     0.0  ...                   1.0       1.0   
36338                     0.0  ...                   1.0       0.0   
74920                     0.0  ...                   1.0       1.0   
...                       ...  ...                   ...       ...   
30545                     0.0  ...                   1.0       0.0   
10020                     0.0  ...                   1.0       0.0   
61559                     0.0  ...                   1.0       1.0   
88900                     0.0  ...                   1.0       0.0   
21891                     0.0  ...                   1.0       1.0   

       MICROCODE_AMD  ISDN_DRV_HISAX  CHARGER_BQ24190  SND_SOC_NAU8825  \
35113            0.0             0.0              0.0              0.0   
12798            0.0             0.0              1.0              0.0   
48733            0.0             0.0              0.0              0.0   
36338            0.0             0.0              0.0              0.0   
74920            0.0             0.0              0.0              0.0   
...              ...             ...              ...              ...   
30545            1.0             0.0              0.0              0.0   
10020            0.0             0.0              0.0              0.0   
61559            1.0             0.0              0.0              0.0   
88900            0.0             0.0              0.0              0.0   
21891            0.0             0.0              1.0              0.0   

       BH1750  NETWORK_FILESYSTEMS  active_options        perf  
35113     0.0                  0.0          1282.0  16618256.0  
12798     0.0                  0.0          1714.0  28190104.0  
48733     0.0                  0.0          2186.0  34274784.0  
36338     0.0                  0.0          1164.0  73377424.0  
74920     0.0                  0.0          1491.0  22122232.0  
...       ...            

In [7]:
num_features = df.shape[1]
num_features

9469

# sample and split the dataset

In [8]:
target_column = 'perf'

In [9]:
y = df[target_column]
X = df.drop(columns=[target_column])

In [10]:
print("X Shape: ",X.shape,"\n Y Shape: ", y.shape)

X Shape:  (1500, 9468) 
 Y Shape:  (1500,)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train_np = X_train.to_numpy(dtype=np.float32)
X_test_np = X_test.to_numpy(dtype=np.float32)
y_train_np = y_train.to_numpy(dtype=np.float32)
y_test_np = y_test.to_numpy(dtype=np.float32)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [14]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


In [15]:
init()

# Put tensors into the Ray object store
X_train_ref = put(X_train_tensor)
y_train_ref = put(y_train_tensor)
X_test_ref = put(X_test_tensor)
y_test_ref = put(y_test_tensor)

2025-01-23 15:06:03,713	INFO worker.py:1821 -- Started a local Ray instance.


# Create a pytorch data loader

In [18]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Create datasets using references
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

# DataLoaders remain the same
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1024, shuffle=True, num_workers=23)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1024, shuffle=False, num_workers=23)

In [ ]:
# Create the models, using pytorch lightningmodule

In [19]:
#torch.set_float32_matmul_precision('medium')

In [20]:
number_of_features = X_train.shape[1]

In [21]:
class LightningModel(L.LightningModule):
    def __init__(self, num_features, activation="ReLU", optimizer_name="Adam", loss_name="MSELoss"):
        super().__init__()
        self.num_features = num_features
        self.activation = activation
        self.optimizer_name = optimizer_name
        self.loss_name = loss_name
        self.model = self.build_model()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = x.view(x.size(0), num_features)
        z = self(x)
        loss = self.get_loss_function()(z, y.unsqueeze(1))
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        x = x.view(x.size(0), num_features)
        z = self(x)
        loss = self.get_loss_function()(z, y.unsqueeze(1))
        self.log('val_loss', loss)
        return loss

    def configure_optimizers(self):
        return self.get_optimizer()

    def get_optimizer(self):
        optimizers = {
            "Adam": torch.optim.Adam(self.parameters(), lr=0.001),
            "AdamW": torch.optim.AdamW(self.parameters(), lr=0.001),
        }
        return optimizers[self.optimizer_name]

    def get_loss_function(self):
        loss_functions = {
            "MSE": nn.MSELoss(),
            "SmoothL1Loss": nn.SmoothL1Loss(),
            "MAE": nn.L1Loss()
        }
        return loss_functions[self.loss_name]

    def get_activation(self):
        activations = {
            "ReLU": nn.ReLU(),
            "PReLU": nn.PReLU(),
            "ELU": nn.ELU()
        }
        return activations[self.activation]

    def build_model(self):
        hidden_size = self.num_features // 2
        hidden_size2 = hidden_size // 2
        return nn.Sequential(
            nn.Linear(self.num_features, hidden_size),
            self.get_activation(),
            nn.Dropout(0.3),
            nn.Linear(hidden_size, hidden_size2),
            self.get_activation(),
            nn.Dropout(0.3),
            nn.Linear(hidden_size2, 1)
        )

change the max epochs if your upper limmit is greater than 100 <br>
`def train_model_tune(config, num_features, train_dataloader, val_dataloader, max_epochs=100):`

In [22]:
def train_model_tune(config, num_features, train_dataloader_ref, val_dataloader_ref, max_epochs=100):

    model = LightningModel(
        num_features=num_features,
        activation=config["activation"],
        optimizer_name=config["optimizer"],
        loss_name=config["loss_function"]
    )

    metrics = {"loss": "val_loss"}
    callbacks = [TuneReportCheckpointCallback(metrics, on="validation_end")]
    
    trainer = L.Trainer(
        max_epochs=max_epochs,
        accelerator='auto',
        devices=1,
        callbacks=callbacks,
        enable_progress_bar=False 
    )
    
    trainer.fit(model, train_dataloader_ref, val_dataloader_ref)


In [23]:


def tune_hyperparameters(num_features, train_dataloader_ref, val_dataloader_ref, num_samples=10):
    # Define the search space
    config = {
        "optimizer": tune.choice(["Adam", "AdamW"]),
        "loss_function": tune.choice(["MAE", "MSE","SmoothL1Loss"]),
        "activation": tune.choice(["ReLU", "PReLU", "ELU"]),
    }
    
    search_algo = OptunaSearch(
        metric="loss",
        mode="min"
    )
    
    scheduler = ASHAScheduler(
        time_attr='training_iteration',
        max_t=30,
        grace_period=10,
        reduction_factor=2
    )
    
    tuner = tune.Tuner(
        tune.with_resources(
            partial(
                train_model_tune,
                num_features=num_features,
                train_dataloader_ref=train_dataloader_ref,
                val_dataloader_ref=val_dataloader_ref
            ),
            resources={"cpu": 16, "gpu": 1}  # Adjust based on your hardware
        ),
        tune_config=tune.TuneConfig(
            metric="loss",
            mode="min",
            search_alg=search_algo,
            scheduler=scheduler,
            num_samples=num_samples
        ),
        param_space=config
    )
    
    results = tuner.fit()
    
    # Get best trial
    best_result = results.get_best_result(metric="loss", mode="min")
    best_trial_config = best_result.config
    best_trial_loss = best_result.metrics['loss']
    print(f"Best trial config: {best_trial_config}")
    print(f"Best trial final validation loss: {best_trial_loss}")
    
    return best_trial_config

In [24]:
num_features = X.shape[1]
num_features

9468

In [ ]:
best_config = tune_hyperparameters(
        num_features=num_features,
        train_dataloader_ref=train_dataloader,
        val_dataloader_ref=test_dataloader,
        num_samples=18
    )

# final_model = LightningModel(
#         num_features=num_features,
#         **best_config
#     )

# final_trainer = L.Trainer(
#         max_epochs=100,
#         accelerator='auto',
#         devices=1
#     )

# final_trainer.fit(final_model, train_dataloader, test_dataloader)


(func pid=35739) GPU available: True (cuda), used: True
(func pid=35739) TPU available: False, using: 0 TPU cores
(func pid=35739) HPU available: False, using: 0 HPUs
(func pid=35739) You are using a CUDA device ('NVIDIA GeForce RTX 4060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
(func pid=35739) LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
(func pid=35739) 
(func pid=35739)   | Name  | Type       | Params | Mode 
(func pid=35739) ---------------------------------------------
(func pid=35739) 0 | model | Sequential | 56.0 M | train
(func pid=35739) ---------------------------------------------
(func pid=35739) 56.0 M    Trainable params
(func pid=35739) 0         Non-trainable params
(func pid=35739) 56.0 M    Total params


2025-01-23 15:06:09,933	ERROR tune_controller.py:1331 -- Trial task failed for trial train_model_tune_4e91198d
Traceback (most recent call last):
  File "/home/pedro/Documents/dl4spl2/venv/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/home/pedro/Documents/dl4spl2/venv/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/home/pedro/Documents/dl4spl2/venv/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/home/pedro/Documents/dl4spl2/venv/lib/python3.10/site-packages/ray/_private/worker.py", line 2755, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeout)
  File "/home/pedro/Documents/dl4spl2/venv/lib/python3.10/site-packages/ray/_private/worker.py", line 906, in get_objects
    raise value.as_instanceof_c

(raylet) The node with node id: 7a950d991cfe0a6e51c57952280b0bde1b7a8053485589e936430a53 and address: 192.168.0.176 and node name: 192.168.0.176 has been marked dead because the detector has missed too many heartbeats from it. This can happen when a 	(1) raylet crashes unexpectedly (OOM, etc.) 
	(2) raylet has lagging heartbeats due to slow network or busy workload.
